In [ ]:
import pandas as pd
import os

pd.options.display.float_format = "{:.2f}".format

listed_directory = "../1.데이터셋/피처생성/상장CROE"
os.listdir(listed_directory)

In [ ]:
result = pd.DataFrame()

for filename in os.listdir(listed_directory):
    df = pd.read_csv(f"../1.데이터셋/피처생성/상장CROE/{filename}", encoding="cp949")
    result = pd.concat([result, df])

In [ ]:
result

In [ ]:
result["회계년도"] = pd.to_datetime(result["회계년도"])
result = result[result["회계년도"].dt.month == 12].reset_index(drop=True)
result["회계년도"] = result["회계년도"].dt.year

In [ ]:
result.head(3)

In [ ]:
len(result['거래소코드'].unique())

#### IFRS(연결) -> IFRS(개별) -> GAAP(연결) -> GAAP(개별)

In [ ]:
print(result.isnull().sum())

In [ ]:
## 자산
result["[U01A100000000]자산(*)(IFRS연결)(천원)"] = result[
    "[U01A100000000]자산(*)(IFRS연결)(천원)"
].fillna(result["[U01A100000000]자산(*)(IFRS)(천원)"])

## 매출액
result["[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"] = result[
    "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"
].fillna(result["[U01B100000000]매출액(수익)(*)(IFRS)(천원)"])

## 자기자본순이익률
result["자기자본순이익률(IFRS연결)"] = result["자기자본순이익률(IFRS연결)"].fillna(result["자기자본순이익률(IFRS)"])

## 종업원수증가율
result["종업원수증가율(IFRS연결)"] = result["종업원수증가율(IFRS연결)"].fillna(result["종업원수증가율(IFRS)"])

## 매출채권회전율
result["매출채권회전률(IFRS연결)"] = result["매출채권회전률(IFRS연결)"].fillna(result["매출채권회전률(IFRS)"])

print(result.isnull().sum())

In [ ]:
## 자산
result["[U01A100000000]자산(*)(IFRS연결)(천원)"] = result[
    "[U01A100000000]자산(*)(IFRS연결)(천원)"
].fillna(result["[U01110000]자산(*)(연결)(천원)"])

## 매출액
result["[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"] = result[
    "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"
].fillna(result["[U01210000]매출액(영업수익)(*)(연결)(천원)"])

## 자기자본순이익률
result["자기자본순이익률(IFRS연결)"] = result["자기자본순이익률(IFRS연결)"].fillna(result["자기자본순이익률"])

## 종업원수증가율
result["종업원수증가율(IFRS연결)"] = result["종업원수증가율(IFRS연결)"].fillna(result["종업원수증가율"])

## 매출채권회전율
result["매출채권회전률(IFRS연결)"] = result["매출채권회전률(IFRS연결)"].fillna(result["매출채권회전률"])

print(result.isnull().sum())

In [ ]:
## 자산
result["[U01A100000000]자산(*)(IFRS연결)(천원)"] = result[
    "[U01A100000000]자산(*)(IFRS연결)(천원)"
].fillna(result["[U01110000]자산(*)(천원)"])

## 매출액
result["[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"] = result[
    "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"
].fillna(result["[U01210000]매출액(영업수익)(*)(천원)"])

print(result.isnull().sum())

In [ ]:
result = result[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)",
        "[U01A100000000]자산(*)(IFRS연결)(천원)",
        "자기자본순이익률(IFRS연결)",
        "매출채권회전률(IFRS연결)",
        "종업원수증가율(IFRS연결)",
		"설립일",
		"상장일"
    ]
]

In [ ]:
result.columns = ['회사명', '거래소코드', '회계년도', '매출액', '총자산', '자기자본순이익률', '매출채권회전률', '종업원수증가율', '설립일', '상장일']

In [ ]:
result

In [ ]:
grouped = result.groupby("거래소코드")

In [ ]:
new_df = pd.DataFrame()

In [ ]:
for code, group in grouped:
    # print(code, group)
    group["CROE"] = group["자기자본순이익률"].diff(1)
    group["총자산변화량"] = group["총자산"].diff(1)
    group["총자산증가율"] = group["총자산변화량"] / group["총자산"].shift(1) * 100
    group["총자산회전율"] = group["매출액"] / group["총자산"]

    new_df = pd.concat([new_df, group], ignore_index=True)

In [ ]:
result.columns

In [ ]:
new_df

In [ ]:
new_df["설립일"] = pd.to_datetime(new_df["설립일"]).dt.year
new_df["영업년수"] = 2024 - new_df["설립일"]
new_df["상장일"] = pd.to_datetime(new_df["상장일"]).dt.year
new_df["상장년수"] = 2024 - new_df["상장일"]

In [ ]:
new_df[["회사명", "거래소코드", "회계년도", '매출채권회전률', '종업원수증가율', "CROE", "총자산증가율", "총자산회전율", '영업년수', '상장년수']].to_csv(
    "../1.데이터셋/피처생성/상장추가피처.csv"
)

In [ ]:
new_df.columns

In [ ]:
new_df.isnull().sum()